In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid

torch.cuda.is_available()

True

In [2]:
dataset = Planetoid("/tmp/Cora", "Cora", split="random", num_train_per_class=int(140/7))
(
    dataset.train_mask.sum().item(),
    dataset.val_mask.sum().item(),
    dataset.test_mask.sum().item()
)
# >>> 1022 nodes are included in the training set

(140, 500, 1000)

In [3]:
(dataset.train_mask & dataset.test_mask).sum().item()

0

In [4]:
dataset.x[dataset.train_mask].shape

torch.Size([140, 1433])

In [5]:
# from https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [7]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.7770
